Some description here in the future! <font color = 'red'> **Still heavily under construction** </font>

In [44]:
import boto3
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.figure_factory as ff

Let's connect to S3 and grab the bucket that contains our data: 

In [5]:
s3 = boto3.client('s3')
bucket = 'personproject.b'

In [12]:
csv_obj = s3.get_object(Bucket=bucket, Key='cahsee2015.txt')
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string), sep = "\t", dtype={"CountyCode": object, "DistrictCode": object, "CharterNumber": object, "SchoolCode": object, "MathTestDate": object, "ELATestDate": object, 
                                                          "ELAPercentNotPassed": float, "GradeLevel": object})

In [13]:
cahsee_15 = df

Record Type: Classifies the level of data 
04=state, 05=county,06=district,07=school

SummaryType: Subgroup

I know I do not want to work with all the data, so I'll examine the columns and pick which ones I'm interested in working with.

In [14]:
cahsee_15.columns.values

array(['CountyCode', 'DistrictCode', 'CharterNumber', 'SchoolCode',
       'CountyName', 'DistrictName', 'SchoolName', 'RecordType',
       'SummaryType', 'Administration', 'MathTestDate',
       'MathGrade10AnswerSheetsProcessed', 'MathNumberTested',
       'MathNumberPassed', 'MathPercentPassed', 'MathNumberNotPassed',
       'MathPercentageNotPassed', 'MathMEanScaleScore',
       'MathPSPercentCorrect', 'MathNSPercentCorrect',
       'MathAFPercentCorrect', 'MathMGPercentCorrect',
       'MathA1PercentCorrect', 'ELATestDate',
       'ELAGrade10AnswerSheetsProcessed', 'ELANumberTested',
       'ELANumberPassed', 'ELAPercentPassed', 'ELANumberNotPassed',
       'ELAPercentNotPassed', 'ELAMeanScaleScore', 'ELAWAPercentCorrect',
       'ELARCPercentCorrect', 'ELALRAPercentCorrect',
       'ELAWSPercentCorrect', 'ELAWCPercentCorrect', 'ELAES1AvgScore',
       'Filler', 'SystemDate', 'GradeLevel', 'MathPercentProfAndAbove',
       'ELAPercentProfAndAbove', 'Unnamed: 42'], dtype=object)

In [10]:
cahsee_15.head(3)

,CountyCode,DistrictCode,CharterNumber,SchoolCode,CountyName,DistrictName,SchoolName,RecordType,SummaryType,Administration,...,ELALRAPercentCorrect,ELAWSPercentCorrect,ELAWCPercentCorrect,ELAES1AvgScore,Filler,SystemDate,GradeLevel,MathPercentProfAndAbove,ELAPercentProfAndAbove,Unnamed: 42
0,01,0,0,0,Alameda County,,,5,1,S,...,53.0,43.0,55.0,2.0,0,07/28/15,12,0.0,0.0,NaN
1,01,0,0,0,Alameda County,,,5,7,S,...,53.0,43.0,55.0,2.0,0,07/28/15,12,0.0,0.0,NaN
2,01,0,0,0,Alameda County,,,5,8,S,...,52.0,43.0,56.0,2.1,0,07/28/15,12,0.0,0.0,NaN


In [ ]:
cahsee_15.info()

In [ ]:
cahsee_15 = cahsee_15[['CountyCode', 'DistrictCode', 'SchoolCode','CountyName','DistrictName', 'SchoolName', 'RecordType', 'SummaryType', 
    'MathGrade10AnswerSheetsProcessed','MathNumberTested', 'MathPercentPassed', 'MathPercentProfAndAbove', 
       'ELAGrade10AnswerSheetsProcessed', 'ELANumberTested', 'ELAPercentPassed', 'ELAPercentProfAndAbove']]

In [ ]:
data_levels = {4: "state", 5: "county", 6: 'district', 7:'school'}

cahsee_15 = cahsee_15.replace({"RecordType": data_levels})

cahsee_15.head(5)

I'm noticing the Percent Prof and Above columns are showing 0% for the first few rows. This appears odd to me as the 'Percent Passed' columns have non-zero values. I want to explore this further after routine data cleaning.

While the report uses dashes to represent suppressed data for number tested (n) <= 10, the file represents supressed data as 0's in the data rows. We'll manually suppress these rows and check the distribution of scores for the test levels.

In [ ]:
cahsee_15.loc[cahsee_15['MathNumberTested'] <= 10, ['MathPercentProfAndAbove','MathPercentPassed']] = None
cahsee_15.loc[cahsee_15['ELANumberTested'] <= 10, ['ELAPercentProfAndAbove','ELAPercentPassed']] = None

In [ ]:
state_level = cahsee_15.loc[cahsee_15['RecordType'] == 'state','MathPercentPassed']
county_level = cahsee_15.loc[cahsee_15['RecordType'] == 'county','MathPercentPassed']
district_level = cahsee_15.loc[cahsee_15['RecordType'] == 'district','MathPercentPassed']
school_level = cahsee_15.loc[cahsee_15['RecordType'] == 'school','MathPercentPassed']

In [ ]:
test = cahsee_15.loc[cahsee_15['MathPercentPassed'] == 0.0, 'MathGrade10AnswerSheetsProcessed']

"In addition to the use of the CAHSEE as a graduation requirement, the CAHSEE
grade ten census administration is used in calculating the Academic Performance
Index for state accountability purposes and Adequate Yearly Progress (AYP) to
meet federal requirements."

In [ ]:
cahsee_15 = (cahsee_15.query('MathGrade10AnswerSheetsProcessed > 0 & ELAGrade10AnswerSheetsProcessed > 0'))

In [ ]:
state_level = cahsee_15.loc[cahsee_15['RecordType'] == 'state','MathPercentPassed']
county_level = cahsee_15.loc[cahsee_15['RecordType'] == 'county','MathPercentPassed']
district_level = cahsee_15.loc[cahsee_15['RecordType'] == 'district','MathPercentPassed']
school_level = cahsee_15.loc[cahsee_15['RecordType'] == 'school','MathPercentPassed']

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.subplot(221)
plt.hist(state_level[~np.isnan(state_level)])
plt.axvline(state_level.mean(), color = 'r', linestyle = 'dashed')
plt.subplot(222)
plt.hist(county_level[~np.isnan(county_level)])
plt.axvline(county_level.mean(), color = 'r', linestyle = 'dashed')
plt.subplot(223)
plt.hist(district_level[~np.isnan(district_level)])
plt.axvline(district_level.mean(), color = 'r', linestyle = 'dashed')
plt.subplot(224)
plt.hist(school_level[~np.isnan(school_level)])
plt.axvline(school_level.mean(), color = 'r', linestyle = 'dashed')

In [ ]:
cahsee_15.loc[cahsee_15['RecordType'] == 'school'].groupby(by = 'CountyName').agg({'MathPercentPassed':'mean', 'ELAPercentPassed': 'mean'}).sort_values('ELAPercentPassed', ascending = False)

In [22]:
ca_directory = pd.read_table("./data/pubschls.txt", sep = "\t", dtype = {"CDSCode" : object, "NCESDist": object, "NCESSchool": object} )
ca_directory.columns.values

array(['CDSCode', 'NCESDist', 'NCESSchool', 'StatusType', 'County',
       'District', 'School', 'Street', 'StreetAbr', 'City', 'Zip',
       'State', 'MailStreet', 'MailStrAbr', 'MailCity', 'MailZip',
       'MailState', 'Phone', 'Ext', 'WebSite', 'OpenDate', 'ClosedDate',
       'Charter', 'CharterNum', 'FundingType', 'DOC', 'DOCType', 'SOC',
       'SOCType', 'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName',
       'GSoffered', 'GSserved', 'Virtual', 'Magnet', 'Latitude',
       'Longitude', 'AdmFName1', 'AdmLName1', 'AdmEmail1', 'AdmFName2',
       'AdmLName2', 'AdmEmail2', 'AdmFName3', 'AdmLName3', 'AdmEmail3',
       'LastUpDate'], dtype=object)

In [35]:
cahsee_15['CDSCode'] = cahsee_15['CountyCode'] + cahsee_15['DistrictCode'] + cahsee_15['SchoolCode']
cahsee_15.loc[cahsee_15['SchoolName'] == 'Alameda High', 'CDSCode'].unique() == ca_directory.loc[ca_directory['School'] == 'Alameda High', 'CDSCode'].unique()

array([ True])

In [42]:
test = pd.merge(cahsee_15, ca_directory.loc[:, ['CDSCode', 'Street', 'City', 'Zip','Latitude', 'Longitude']], 
                left_on = 'CDSCode', right_on = 'CDSCode', how = 'inner')
cahsee_15.shape[0] - test.shape[0]

51682